In [ ]:
%pylab inline

In [ ]:
import ephem
import gzip

tau = pi * 2.0
white = (1.0, 1.0, 1.0)

In [ ]:
f = gzip.open('data/ELEMENTS.NUMBR.gz')
lines = iter(f)
header = next(lines)
dashes = next(lines)

bodies = []

for line in f:
    fields = line[24:].split()
    b = ephem.EllipticalBody()
    b._epoch = int(fields[0])     # "Epoch"
    b._a = float(fields[1])       # "a"
    b._e = float(fields[2])       # "e"
    b._inc = float(fields[3])     # "i"
    b._om = float(fields[4])      # "w"
    b._Om = float(fields[5])      # "Node"
    b._M = float(fields[6])       # "M"
    b._epoch_M = b._epoch  # complete guess
    b._g = float(fields[7])
    bodies.append(b)
    
print 'Loaded', len(bodies), 'asteroids'

How did the number of asteroids known to our species become so large?
Because since the first LINEAR system started operating in 1998,
asteroid discovery has been an automated process
that now involves dozens of automated telescopes across the world.

Here is a video animation of our asteroid discoveries so far:

http://www.youtube.com/watch?v=S_d-gs0WoUw

In [ ]:
def compute_asteroid_positions(mjd):
    for b in bodies:
        b.compute(mjd)
    theta = np.array([ b.hlon for b in bodies ])
    r = np.array([ b.sun_distance for b in bodies ])
    return theta, r

In [ ]:
mjd = ephem.Date('2013/1/11 15:45')
theta, r = compute_asteroid_positions(mjd)

print 'First asteroid is at angle', theta[0]
print 'and at distance', r[0], 'AU from the Sun'

In [ ]:
def generate_planet_positions(cls, mjd0, mjd1):
    planet = cls()
    dates = np.arange(mjd0, mjd1, 1.0)
    for date in dates:
        planet.compute(date)
        yield (planet.hlon, planet.sun_distance)

In [ ]:
subplot = figure().add_subplot(111, polar=True)

subplot.scatter(theta, r, s=0.5)

for cls in ephem.Jupiter, ephem.Mars, ephem.Saturn:
    g = generate_planet_positions(cls, mjd - 30, mjd + 30)
    theta_r = np.array(list(g)).T
    subplot.plot(theta_r[0], theta_r[1], linewidth=8)

subplot.axis([None, None, 0.0, 12.0])

In [ ]:
x =  r * cos(theta)
y = -r * sin(theta)

heatmap, xedges, yedges = np.histogram2d(
    y, x, bins=200, range=[[-7, 7], [-7, 7]])

extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
imshow(heatmap, extent=extent)

In [ ]:
imshow(log(heatmap + 1.0), extent=extent)

In [ ]:
n, bins, patches = hist(r, bins=60, range=[0.0, 12.0])

for cls in ephem.Mars, ephem.Jupiter, ephem.Saturn:
    planet = cls()
    planet.compute(mjd)
    annotate(
        planet.name,
        xy=(planet.sun_distance, 30000),
        xytext=(planet.sun_distance, 50000),
        arrowprops=dict(facecolor='black', shrink=0.2),
        )

In [ ]:
jupiter = ephem.Jupiter()
jupiter.compute(mjd)

print 'Jupiter heliocentric longitude =', jupiter.hlon

In [ ]:
n, bins, patches = hist(theta, bins=48)
jpos = (jupiter.hlon, 1500.0)
annotate('J', xy=jpos, fontsize=24, color=white)
axis([0.0, tau, None, None])

In [ ]:
n, bins, patches = hist(theta[r < 3.0], bins=48)
jpos = (jupiter.hlon, 850.0)
annotate('J', xy=jpos, fontsize=24, color=white)
axis([0.0, tau, None, None])

In [ ]:
n, bins, patches = hist(theta[r > 4.5], bins=48)
jpos = (jupiter.hlon, 200.0)
annotate('J', xy=jpos, fontsize=24)
axis([0.0, tau, None, None])